In [3]:
#include "TH2.h"
#include "TStyle.h"
#include "TCanvas.h"
#include <stdio.h>
#include <iostream>
#include "TGraph.h"
#include "TGraphErrors.h"
#include "TFile.h"
#include "TLine.h"
#include <TF1.h>
#include <TVector.h>

void mean(int run = 3965, int nImage=100){
  gDirectory->DeleteAll();
  gStyle->SetPalette(0);
  FILE *dataC1;
  float img;
  int row, col, rowBin, colBin, i;
  int nRow = 2048;
  int nCol = 2048;
  int iImg;
  int intBinSize = 4;
  int nBinX = nRow/intBinSize;
  int nBinY = nCol/intBinSize;
  float light[30];
  float pedestal[30];

  TH2F *hImagePed      = new TH2F("hImagePed",     "",nCol,0,nCol,nRow,0,nRow);
  TH2F *hImageRaw       = new TH2F("hImageRaw",      "",nCol,0,nCol,nRow,0,nRow);
  
  char name[1000], nameImg[1000], outname[1000];
  double value;
  //sprintf(name,"https://swift.cloud.infn.it:8080/v1/AUTH_1e60fe39fba04701aa5ffc0b97871ed8/Cygnus/Data/LAB/histograms_Run%05d.root", run);
  sprintf(name,"https://s3.cloud.infn.it/v1/AUTH_2ebf769785574195bde2ff418deac08a/cygnus/Data/LAB/histograms_Run%05d.root",run);
  TFile *f = TFile::Open(name);
  if(f!=NULL) {
    //    if(f->IsOpen()) {
    cout<<" File "<<name<<" with size "<<f->GetSize() <<" opened!"<<endl;
  } else {
    cout<<" File "<<name<<" does not exists!"<<endl;
    return;
  }
  for(row=1; row<=nRow; row++){
    for(col=1; col<=nCol; col++){
      hImagePed->SetBinContent(col, row, 0);
    }
  }
  for(img=0; img<nImage; img++){
    hImageRaw->Reset();
    sprintf(nameImg,"pic_run%05d_ev%.0f", run, img);  
    hImageRaw = (TH2F*)gDirectory->Get(nameImg)->Clone();;
    hImageRaw->SetDirectory(0);
    for(row=1; row<=nRow; row++){
      for(col=1; col<=nCol; col++){
	hImagePed->SetBinContent(col, row, (hImagePed->GetBinContent(col, row))+(1.0/nImage)*hImageRaw->GetBinContent(col, row));
      }
    }
  }
  f->Close();

sprintf(outname,"./ped_run%d.root",run);
 TFile *outfile = new TFile(outname, "RECREATE");
 hImagePed     ->Write();
 outfile        ->Write();
 outfile        ->Close();
}



In [4]:
mean()

 File https://s3.cloud.infn.it/v1/AUTH_2ebf769785574195bde2ff418deac08a/cygnus/Data/LAB/histograms_Run03965.root with size 273404549 opened!
